In [1]:
import os
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import numpy as np
import flopy

In [2]:

# Load observed and synthetic data
observed_data = pd.read_csv('C:\\Users\\sebas\\Documents\\UGent\\Sem 3\\Thesis\\GondebeekModel\\Outputs\\observed_data.csv')
synthetic_data = pd.read_csv('C:\\Users\\sebas\\Documents\\UGent\\Sem 3\\Thesis\\GondebeekModel\\Outputs\\synthetic_data.csv')

# Get the observation points that are in either observed or synthetic data
obs_points = set(observed_data['obs_point']).union(synthetic_data['obs_point'])

# Load all .smp files from the directory
directory = 'C:\\Users\\sebas\\Documents\\UGent\\Sem 3\\Thesis\\GondebeekModel\\Outputs'
simulated_data = {}
for filename in os.listdir(directory):
    if filename.endswith('.smp'):
        iteration = filename.split('_')[1].split('.')[0]  # Get the iteration number from the filename
        data = pd.read_csv(os.path.join(directory, filename), sep='\s+', names=['obs_point', 'date', 'time', 'head'])
        data = data[data['obs_point'].isin(obs_points)]  # Filter the data to only include the observation points in obs_points
        simulated_data[f'Simulated {iteration}'] = data

# Update the dropdown menu
dropdown = widgets.Dropdown(
    options=list(obs_points),
    description='Obs Point:',
)

def plot_data(obs_point):
    # Plot the data for the selected obs_point
    plt.figure(figsize=(10, 6))
    
    # Initialize min_head and max_head
    min_head = float('inf')
    max_head = float('-inf')
        
    # Plot simulated data
    for label, data in simulated_data.items():
        data_to_plot = data[data['obs_point'] == obs_point]
        dates = pd.to_datetime(data_to_plot['date'], format='%d/%m/%Y').to_numpy()
        plt.plot(dates, data_to_plot['head'].to_numpy(), label=label)
        min_head = min(min_head, data_to_plot['head'].min())
        max_head = max(max_head, data_to_plot['head'].max())
        
    # Plot and highlight observed data
    data_to_plot = observed_data[observed_data['obs_point'] == obs_point]
    dates = pd.to_datetime(data_to_plot['date'], format='mixed').to_numpy()
    plt.plot(dates, data_to_plot['head'].to_numpy(), label='Observed', linestyle='--', color='blue')
    plt.scatter(dates, data_to_plot['head'].to_numpy(), color='blue')
    min_head = min(min_head, data_to_plot['head'].min())
    max_head = max(max_head, data_to_plot['head'].max())
    
    # Plot and highlight synthetic data
    data_to_plot = synthetic_data[synthetic_data['obs_point'] == obs_point]
    dates = pd.to_datetime(data_to_plot['date'], format='mixed').to_numpy()
    plt.plot(dates, data_to_plot['head'].to_numpy(), label='Synthetic', linestyle=':', color='orange')
    plt.scatter(dates, data_to_plot['head'].to_numpy(), color='orange')
    min_head = min(min_head, data_to_plot['head'].min())
    max_head = max(max_head, data_to_plot['head'].max())

    # Set the y-axis limits
    plt.ylim(min_head * 0.8, max_head * 1.2)

    plt.title(f'Head for Obs Point {obs_point}')
    plt.xlabel('Date')
    plt.ylabel('Head')
    plt.legend()
    plt.show()
# Create an interactive plot with the dropdown widget
interact(plot_data, obs_point=dropdown)

<>:14: DeprecationWarning: invalid escape sequence \s
<>:14: DeprecationWarning: invalid escape sequence \s
C:\Users\sebas\AppData\Local\Temp\ipykernel_4244\421470249.py:14: DeprecationWarning: invalid escape sequence \s
  data = pd.read_csv(os.path.join(directory, filename), sep='\s+', names=['obs_point', 'date', 'time', 'head'])
c:\Users\sebas\anaconda3\envs\BastianMod\lib\site-packages\ipywidgets\widgets\interaction.py:43: DeprecationWarning: `ipykernel.pylab.backend_inline` is deprecated, directly use `matplotlib_inline.backend_inline`
  from ipykernel.pylab.backend_inline import flush_figures


interactive(children=(Dropdown(description='Obs Point:', options=('MOORT_8', 'OOST_17', 'MEL_4', 'MOORT_7', 'M…

<function __main__.plot_data(obs_point)>

Load the model once

In [3]:
# Define the workspace and model name
model_ws = 'C:\\Users\\sebas\\Documents\\UGent\\Sem 3\\Thesis\\GondebeekModel\\Modflow_Master'
model_name = 'Gondebeek'

# Create a simulation
sim = flopy.mf6.MFSimulation.load(sim_name=model_name, exe_name='mf6', sim_ws=model_ws)

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package dis...
    loading package drn...
    loading package riv...
    loading package rch...
    loading package sto...
    loading package ic...
    loading package oc...
    loading package npf...
  loading solution package gondebeek...


In [4]:


# Get the groundwater flow model
gwf = sim.get_model(model_name)

# Get the hydraulic conductivity values
hk = gwf.npf.k.array

# Get the idomain values
idomain = gwf.dis.idomain.array

# Mask the hydraulic conductivity values with the idomain
hk_masked = np.ma.masked_where(idomain == 0, hk)

# Get unique hydraulic conductivity values and assign each a unique random color
unique_hk = np.unique(hk_masked.compressed())  # Use the compressed() method to remove the MaskedConstant values
colors = [np.random.rand(3,) for _ in range(len(unique_hk))]
hk_color_dict = dict(zip(unique_hk, colors))

# Read the observation points
obs_points = pd.read_csv(model_ws + '/bore_coord.dat', sep='\t', header=None, names=['name', 'x', 'y', 'layer'])

# Convert the x and y coordinates to row and column indices
obs_points['row'] = np.floor((gwf.modelgrid.extent[3] - obs_points['y']) / gwf.modelgrid.delc[0]).astype(int)
obs_points['col'] = np.floor((obs_points['x'] - gwf.modelgrid.extent[0]) / gwf.modelgrid.delr[0]).astype(int)

# Create a dropdown menu for selecting the observation point
dropdown = widgets.Dropdown(options=obs_points['name'], description='Obs Point:')

# Define a function to plot the data for the selected observation point
def plot_data(obs_point):
    # Clear the current figure
    plt.clf()
    plt.figure(figsize=(15, 10))
    # Get the layer of the selected observation point
    layer = obs_points[obs_points['name'] == obs_point]['layer'].values[0] - 1
    # Create a dictionary where each unique hydraulic conductivity value is a key and the corresponding value is a list of coordinates where this hydraulic conductivity value occurs
    hk_coords = {hk: [] for hk in unique_hk}
    for j in range(hk_masked[layer].shape[0]):
        for k in range(hk_masked[layer].shape[1]):
            if idomain[layer, j, k] != 0:  # Only consider active cells
                hk_coords[hk_masked[layer, j, k]].append((j, k))

# For each unique hydraulic conductivity value, calculate the center of the corresponding coordinates and display the hydraulic conductivity value at this center
    for hk, coords in hk_coords.items():
        if coords:  # If the list of coordinates is not empty
            center = np.mean(coords, axis=0).astype(int)  # Calculate the center of the coordinates
            plt.text(center[1], center[0], f'{hk:.3f}', ha='center', va='center', color='black', bbox=dict(facecolor='white', alpha=0.5, edgecolor='white'))  # Use three significant digits
            
    # Plot the hydraulic conductivity of the layer
    color_layer = np.ones((hk_masked[layer].shape[0], hk_masked[layer].shape[1], 3))  # Initialize the color_layer array with ones
    for j in range(hk_masked[layer].shape[0]):
        for k in range(hk_masked[layer].shape[1]):
            if idomain[layer, j, k] != 0:  # Only consider active cells
                color_layer[j, k] = hk_color_dict[hk_masked[layer, j, k]]
    plt.imshow(color_layer)
    
    plt.title(f'Hydraulic Conductivity Layer {layer+1} [m/d]')
    
    # Plot the observation points on the current layer
    layer_obs_points = obs_points[obs_points['layer'] == layer+1]
    plt.scatter(layer_obs_points['col'], layer_obs_points['row'], color='blue')
    
    # Highlight the selected observation point in red
    selected_obs_point = layer_obs_points[layer_obs_points['name'] == obs_point]
    plt.scatter(selected_obs_point['col'], selected_obs_point['row'], color='red')
    
    plt.show()

# Create an interactive widget to select the observation point
widgets.interact(plot_data, obs_point=dropdown)

interactive(children=(Dropdown(description='Obs Point:', options=('aelm_1', 'aelm_2', 'aelm_3', 'Mel_1', 'Mel_…

<function __main__.plot_data(obs_point)>

In [5]:
import ipywidgets as widgets
from ipywidgets import interact, HBox
import matplotlib.pyplot as plt
import pandas as pd
import os

# Load observed and synthetic data
observed_data = pd.read_csv('observed_data.csv')
synthetic_data = pd.read_csv('synthetic_data.csv')

# Get the observation points that are in either observed or synthetic data
obs_points = set(observed_data['obs_point']).union(synthetic_data['obs_point'])

# Load all .smp files from the directory
directory = 'C:\\Users\\sebas\\Documents\\UGent\\Sem 3\\Thesis\\GondebeekModel\\Outputs'
simulated_data = {}
for filename in os.listdir(directory):
    if filename.endswith('.smp'):
        iteration = filename.split('_')[1].split('.')[0]
        data = pd.read_csv(os.path.join(directory, filename), sep='\s+', names=['obs_point', 'date', 'time', 'head'])
        data = data[data['obs_point'].isin(obs_points)]
        simulated_data[f'Simulated {iteration}'] = data

# Get the groundwater flow model
gwf = sim.get_model(model_name)

# Get the hydraulic conductivity values
hk = gwf.npf.k.array

# Get the idomain values
idomain = gwf.dis.idomain.array

# Mask the hydraulic conductivity values with the idomain
hk_masked = np.ma.masked_where(idomain == 0, hk)

# Get unique hydraulic conductivity values and assign each a unique random color
unique_hk = np.unique(hk_masked.compressed())
colors = [np.random.rand(3,) for _ in range(len(unique_hk))]
hk_color_dict = dict(zip(unique_hk, colors))

# Read the observation points for hydraulic conductivity
hk_obs_points = pd.read_csv(model_ws + '/bore_coord.dat', sep='\t', header=None, names=['name', 'x', 'y', 'layer'])

# Convert the x and y coordinates to row and column indices
hk_obs_points['row'] = np.floor((gwf.modelgrid.extent[3] - hk_obs_points['y']) / gwf.modelgrid.delc[0]).astype(int)
hk_obs_points['col'] = np.floor((hk_obs_points['x'] - gwf.modelgrid.extent[0]) / gwf.modelgrid.delr[0]).astype(int)

obs_points = set(obs_point.upper() for obs_point in observed_data['obs_point']).union(obs_point.upper() for obs_point in synthetic_data['obs_point'])
hk_obs_points['name'] = hk_obs_points['name'].str.upper()

# Now perform the intersection
common_obs_points = obs_points.intersection(set(hk_obs_points['name']))


# Update the dropdown menu
dropdown = widgets.Dropdown(
    options=sorted(list(common_obs_points)),
    description='Obs Point:',
)

# Define the plotting functions
def plot_head_data(obs_point):
    plt.clf()
    # Plot the data for the selected obs_point
    plt.figure(figsize=(10, 6))
    
    # Initialize min_head and max_head
    min_head = float('inf')
    max_head = float('-inf')
        
    # Plot simulated data
    for label, data in simulated_data.items():
        data_to_plot = data[data['obs_point'] == obs_point]
        dates = pd.to_datetime(data_to_plot['date'], format='%d/%m/%Y').to_numpy()
        plt.plot(dates, data_to_plot['head'].to_numpy(), label=label)
        min_head = min(min_head, data_to_plot['head'].min())
        max_head = max(max_head, data_to_plot['head'].max())
        
    # Plot and highlight observed data
    data_to_plot = observed_data[observed_data['obs_point'] == obs_point]
    dates = pd.to_datetime(data_to_plot['date'], format='mixed').to_numpy()
    plt.plot(dates, data_to_plot['head'].to_numpy(), label='Observed', linestyle='--', color='blue')
    plt.scatter(dates, data_to_plot['head'].to_numpy(), color='blue')
    min_head = min(min_head, data_to_plot['head'].min())
    max_head = max(max_head, data_to_plot['head'].max())
    
    # Plot and highlight synthetic data
    data_to_plot = synthetic_data[synthetic_data['obs_point'] == obs_point]
    dates = pd.to_datetime(data_to_plot['date'], format='mixed').to_numpy()
    plt.plot(dates, data_to_plot['head'].to_numpy(), label='Synthetic', linestyle=':', color='orange')
    plt.scatter(dates, data_to_plot['head'].to_numpy(), color='orange')
    min_head = min(min_head, data_to_plot['head'].min())
    max_head = max(max_head, data_to_plot['head'].max())

    # Set the y-axis limits
    plt.ylim(min_head * 0.8, max_head * 1.2)

    plt.title(f'Head for Obs Point {obs_point}')
    plt.xlabel('Date')
    plt.ylabel('Head')
    plt.legend()
    plt.show()
def plot_hk_data(obs_point):
    # Clear the current figure
    plt.clf()
    plt.figure(figsize=(15, 10))
    # Get the layer of the selected observation point
    layer = hk_obs_points[hk_obs_points['name'] == obs_point]['layer'].values[0] - 1
    # Create a dictionary where each unique hydraulic conductivity value is a key and the corresponding value is a list of coordinates where this hydraulic conductivity value occurs
    hk_coords = {hk: [] for hk in unique_hk}
    for j in range(hk_masked[layer].shape[0]):
        for k in range(hk_masked[layer].shape[1]):
            if idomain[layer, j, k] != 0:  # Only consider active cells
                hk_coords[hk_masked[layer, j, k]].append((j, k))

# For each unique hydraulic conductivity value, calculate the center of the corresponding coordinates and display the hydraulic conductivity value at this center
    for hk, coords in hk_coords.items():
        if coords:  # If the list of coordinates is not empty
            center = np.mean(coords, axis=0).astype(int)  # Calculate the center of the coordinates
            plt.text(center[1], center[0], f'{hk:.3f}', ha='center', va='center', color='black', bbox=dict(facecolor='white', alpha=0.5, edgecolor='white'))  # Use three significant digits
            
    # Plot the hydraulic conductivity of the layer
    color_layer = np.ones((hk_masked[layer].shape[0], hk_masked[layer].shape[1], 3))  # Initialize the color_layer array with ones
    for j in range(hk_masked[layer].shape[0]):
        for k in range(hk_masked[layer].shape[1]):
            if idomain[layer, j, k] != 0:  # Only consider active cells
                color_layer[j, k] = hk_color_dict[hk_masked[layer, j, k]]
    plt.imshow(color_layer)
    
    plt.title(f'Hydraulic Conductivity Layer {layer+1} [m/d]')
    
    # Plot the observation points on the current layer
    layer_obs_points = hk_obs_points[(hk_obs_points['layer'] == layer+1) & (hk_obs_points['name'].isin(common_obs_points))]
    plt.scatter(layer_obs_points['col'], layer_obs_points['row'], color='blue')
    
    # Highlight the selected observation point in red
    selected_obs_point = layer_obs_points[layer_obs_points['name'] == obs_point]
    plt.scatter(selected_obs_point['col'], selected_obs_point['row'], color='red')
    
    plt.show()

# Define a function to plot both datasets side by side
def plot_both(obs_point):
    plt.subplot(1, 2, 1)
    plot_head_data(obs_point)
    plt.subplot(1, 2, 2)
    plot_hk_data(obs_point)
    plt.show()

# Create an interactive plot with the dropdown widget
interact(plot_both, obs_point=dropdown)


<>:20: DeprecationWarning: invalid escape sequence \s
<>:20: DeprecationWarning: invalid escape sequence \s
C:\Users\sebas\AppData\Local\Temp\ipykernel_4244\4155020184.py:20: DeprecationWarning: invalid escape sequence \s
  data = pd.read_csv(os.path.join(directory, filename), sep='\s+', names=['obs_point', 'date', 'time', 'head'])


interactive(children=(Dropdown(description='Obs Point:', options=('MEL_1', 'MEL_2', 'MEL_3', 'MEL_4', 'MER_5',…

<function __main__.plot_both(obs_point)>